In [10]:
import plotly.express as px
import pandas as pd
import segyio
import numpy as np
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression

In [2]:
"""
Алгоритм:
1) куб переделывается в словарь {координаты по инлайнам и крослайнам:[матожидание, дисперсия сейсмотрассы]}
2) Разделяем словарь согласно границам сшивки, которые передаются из другого теста.
3) Словари переводим в списки для отображения крос-плота
4) строим крос-плот и сраниваем отклонения линий тренда. Если отличие больше на 10 % то тест не пройден


В сшитых кубах границы по Икслайнам 100 (середина), по Инлайнам нету
"""

'\nАлгоритм:\n1) куб переделывается в словарь {координаты по инлайнам и крослайнам:[матожидание, дисперсия сейсмотрассы]}\n2) Разделяем словарь согласно границам сшивки, которые передаются из другого теста.\n3) Словари переводим в списки для отображения крос-плота\n4) строим крос-плот и сраниваем отклонения линий тренда. Если отличие больше на 15 % то тест не пройден\n\n\nВ сшитых кубах границы по Икслайнам 100 (середина), по Инлайнам нету\n'

In [2]:
"""
Тут будем работать с шитыми кубами у которых нет кровли и подошвы и разломов. Поэтому ридер пока мой. (напиши мне, чтобы я скинул сшитые кубы)
Нужно будет потом из датафрейма сделать словарь как в функции
"""
def read_seismic_cube(file_path):
    """
    :param file_path: Путь к файлу
    :return: возвращает куб в виде списков по Inlines, Xlines и Sampels (координаты x, y, z)
    а также словарь, где ключ это координата пересечения Inlines и Xlines, а значение это список матожидание и дисперсия сейсмотрассы
    """
    with segyio.open(file_path, 'r') as segyfile:
    	x = list(segyfile.ilines)
    	y = list(segyfile.xlines)
    	z = list(segyfile.samples)
    	cube = segyio.tools.cube(segyfile)
    	M_D = {f"{x[i]}_{y[j]}": [np.mean(cube[i][j][668:1340]), np.var(cube[i][j][668:1340])]  for i in range(len(x)) for j in range(len(y))}
    return x, y, z, M_D

In [3]:
def inform_from_cube(cube1, cube2, index):
    Y_TWT = []
    X_TVDSS = []
    for key in cube1:
        Y_TWT.append(cube1[key][index])
        X_TVDSS.append(cube2[key][index])
    return Y_TWT, X_TVDSS

In [4]:
file_path = 'C:/HV/Seismic/datas/shitie_cubs_TWT.segy'
ilines_TWT, xlines_TWT, samples_TWT, M_D_TWT = read_seismic_cube(file_path)

file_path = 'C:/HV/Seismic/datas/Depth_shiti_cub_v_glubine.segy'
ilines_TVDSS, xlines_TVDSS, samples_TVDSS, M_D_TVDSS = read_seismic_cube(file_path)


In [5]:
granica_xline_index = 100
granica_iline_index = 0

cube_TWT_1 = {}
cube_TWT_2 = {}
cube_TVDSS_1 = {}
cube_TVDSS_2 = {}
if granica_xline_index!=0:
    for i in range(len(ilines_TWT)):
        for j in range(granica_xline_index):
            cube_TWT_1[f"{ilines_TWT[i]}_{xlines_TWT[j]}"] = M_D_TWT[f"{ilines_TWT[i]}_{xlines_TWT[j]}"]
            cube_TVDSS_1[f"{ilines_TVDSS[i]}_{xlines_TVDSS[j]}"] = M_D_TVDSS[f"{ilines_TVDSS[i]}_{xlines_TVDSS[j]}"]
        for j in range(granica_xline_index, len(xlines_TWT)):
            cube_TWT_2[f"{ilines_TWT[i]}_{xlines_TWT[j]}"] = M_D_TWT[f"{ilines_TWT[i]}_{xlines_TWT[j]}"]
            cube_TVDSS_2[f"{ilines_TVDSS[i]}_{xlines_TVDSS[j]}"] = M_D_TVDSS[f"{ilines_TVDSS[i]}_{xlines_TVDSS[j]}"]
elif granica_iline_index != 0:
    for j in range(len(xlines_TWT)):
    
        for i in range(granica_iline_index):
            cube_TWT_1[f"{ilines_TWT[i]}_{xlines_TWT[j]}"] = M_D_TWT[f"{ilines_TWT[i]}_{xlines_TWT[j]}"]
            cube_TVDSS_1[f"{ilines_TVDSS[i]}_{xlines_TVDSS[j]}"] = M_D_TVDSS[f"{ilines_TVDSS[i]}_{xlines_TVDSS[j]}"]
        for i in range(granica_iline_index, len(ilines_TWT)):
            cube_TWT_2[f"{ilines_TWT[i]}_{xlines_TWT[j]}"] = M_D_TWT[f"{ilines_TWT[i]}_{xlines_TWT[j]}"]
            cube_TVDSS_2[f"{ilines_TVDSS[i]}_{xlines_TVDSS[j]}"] = M_D_TVDSS[f"{ilines_TVDSS[i]}_{xlines_TVDSS[j]}"]
    
            

In [6]:
Y_M_TWT_1, X_M_TVDSS_1 = inform_from_cube(cube_TWT_1, cube_TVDSS_1, 0)
Y_M_TWT_2, X_M_TVDSS_2 = inform_from_cube(cube_TWT_2, cube_TVDSS_2, 0)

In [21]:
# Функция для проверки сильной линейной зависимости
def check_strong_linear_dependency(x, y, threshold=0.9):
    model = LinearRegression()
    model.fit(x.reshape(-1, 1), y)
    r_squared = model.score(x.reshape(-1, 1), y)
    return r_squared >= threshold

In [24]:
#Поиск коэффициента наклона тренда при сильной линейной зависимости
def koef_naklon (x, y):
    # Преобразование в двумерный массив для sklearn
    x_reshaped = x.reshape(-1, 1)
    
    # Создаем модель линейной регрессии
    model = LinearRegression()
    
    # Обучаем модель
    model.fit(x_reshaped, y)
    
    # Получаем наклон (коэффициент) и пересечение
    m = model.coef_[0]  # Наклон
    b = model.intercept_  # Пересечение
    
    return m

In [31]:
def plot_with_trend_lines(x1, y1, x2, y2):

    

    # Преобразование в массивы NumPy
    x1 = np.array(x1)
    y1 = np.array(y1)
    x2 = np.array(x2)
    y2 = np.array(y2)

    fig = go.Figure()

    # Проверка на сильную линейную зависимость для куба 1
    strong_dependency_1 = check_strong_linear_dependency(x1, y1)
    
    # Добавляем точки для первой пары (x1, y1)
    fig.add_trace(go.Scatter(x=x1, y=y1, mode='markers+lines', name='Значения для куба 1', marker=dict(color='red')))
    
    # Добавляем линии тренда для первой пары (если не отключено)
    
    if not strong_dependency_1:
        m1, b1 = np.polyfit(x1, y1, 1)
        trend1 = m1 * x1 + b1
        fig.add_trace(go.Scatter(x=x1, y=trend1, mode='lines', name='Линия тренда для куба 1', line=dict(color='red', dash='dash')))
    else:
        m1 = koef_naklon(x1, y1)
    
    # Проверка на сильную линейную зависимость для куба 2
    strong_dependency_2 = check_strong_linear_dependency(x2, y2)

    # Добавляем точки для второй пары (x2, y2)
    fig.add_trace(go.Scatter(x=x2, y=y2, mode='markers+lines', name='Значения для куба 2', marker=dict(color='blue')))
    
    # Добавляем линии тренда для второй пары (если не отключено)
    if not strong_dependency_2:
        m2, b2 = np.polyfit(x2, y2, 1)
        trend2 = m2 * x2 + b2
        fig.add_trace(go.Scatter(x=x2, y=trend2, mode='lines', name='Линия тренда для куба 2', line=dict(color='blue', dash='dash')))
    else:
        m2 = koef_naklon(x2, y2)
    
    print(m1, m2)
    percentage_difference = round((max(m1, m2) - min(m1, m2)) * 100 / max(m1, m2), 3)
   
    # Настройка графика
    fig.update_layout(
        title='Сравнение пар данных',
        xaxis_title='TVD',
        yaxis_title='TWT',
        plot_bgcolor='black',  # Черный фон
        paper_bgcolor='black',  # Черный фон для бумаги
        font_color='white',  # Цвет шрифта
        legend=dict(x=0, y=1),
        template='plotly_dark'  # Темная тема
    )

    # Показываем график
    fig.show()
    return percentage_difference


In [32]:
procent = plot_with_trend_lines(X_M_TVDSS_1, Y_M_TWT_1, X_M_TVDSS_2, Y_M_TWT_2)
print(f"Процент отличия трендов: {procent} %")
if procent <= 10:
    print('Несмотря на несбалансированные амплитуды сшитых кубов, тренд время-глубина однородный, можно проводить структурную интерпретацию')
else:
    print('Тренд время-глубина разделяется на несколько зон. Будьте внимательны при структурной интерпретации')



1249.9917 1249.9877


Процент отличия трендов: 0.0 %
Несмотря на несбалансированные амплитуды сшитых кубов, тренд время-глубина однородный, можно проводить структурную интерпретацию
